<a href="https://colab.research.google.com/github/ShreshthSaxena/Club-Mahindra-DataOlympics/blob/master/Club_Mahindra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [0]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelBinarizer as lb
from sklearn.metrics import mean_squared_error

In [0]:
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [16]:
df.fillna({"state_code_residence":8.0, "season_holidayed_code":2.0}, inplace=True)
df.isnull().sum()

reservation_id                        0
booking_date                          0
checkin_date                          0
checkout_date                         0
channel_code                          0
main_product_code                     0
numberofadults                        0
numberofchildren                      0
persontravellingid                    0
resort_region_code                    0
resort_type_code                      0
room_type_booked_code                 0
roomnights                            0
season_holidayed_code                 0
state_code_residence                  0
state_code_resort                     0
total_pax                             0
member_age_buckets                    0
booking_type_code                     0
memberid                              0
cluster_code                          0
reservationstatusid_code              0
resort_id                             0
amount_spent_per_room_night_scaled    0
dtype: int64

In [8]:
for i in df.columns:
   print('column:',i,'len:',len(set(df[i])))

column: reservation_id len: 341424
column: booking_date len: 1621
column: checkin_date len: 1532
column: checkout_date len: 1531
column: channel_code len: 3
column: main_product_code len: 5
column: numberofadults len: 28
column: numberofchildren len: 12
column: persontravellingid len: 6
column: resort_region_code len: 3
column: resort_type_code len: 7
column: room_type_booked_code len: 6
column: roomnights len: 44
column: season_holidayed_code len: 4
column: state_code_residence len: 37
column: state_code_resort len: 11
column: total_pax len: 18
column: member_age_buckets len: 10
column: booking_type_code len: 2
column: memberid len: 101327
column: cluster_code len: 6
column: reservationstatusid_code len: 4
column: resort_id len: 32
column: amount_spent_per_room_night_scaled len: 278083


In [17]:
df['checkin_date']= pd.to_datetime(df['checkin_date'],dayfirst = True)
df['checkout_date']= pd.to_datetime(df['checkout_date'],dayfirst = True)
df['booking_date']= pd.to_datetime(df['booking_date'],dayfirst = True)
df['stay']= df['checkout_date'] - df['checkin_date']
df['stay']= df['stay'].dt.days
df['planning']= df['checkin_date'] - df['booking_date']
df['planning']= df['planning'].dt.days
df['extradays']= df['roomnights'] - df['stay']

df = df.drop(['checkin_date','checkout_date','booking_date','roomnights'],1)
df.dtypes

#'reservation_id','memberid','resort_id'

reservation_id                         object
channel_code                            int64
main_product_code                       int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
season_holidayed_code                 float64
state_code_residence                  float64
state_code_resort                       int64
total_pax                               int64
member_age_buckets                     object
booking_type_code                       int64
memberid                               object
cluster_code                           object
reservationstatusid_code               object
resort_id                              object
amount_spent_per_room_night_scaled    float64
stay                                    int64
planning                          

In [18]:
for i in df.index:
  if df.iloc[i]['total_pax'] == 0:
    df['total_pax'][i] = df.iloc[i]['numberofadults']+df.iloc[i]['numberofchildren'] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df['adultsp'] = df['numberofadults']/df['total_pax']
df['childrenp'] = df['numberofchildren']/df['total_pax']
df.describe()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,season_holidayed_code,state_code_residence,state_code_resort,total_pax,booking_type_code,amount_spent_per_room_night_scaled,stay,planning,extradays,adultsp,childrenp
count,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000
mean,1.902810,2.208738,3.275361,0.362696,210.288905,1.772960,2.193832,2.890321,2.673637,7.271024,4.763303,3.193519,1.124209,7.720801,2.466646,46.084754,1.271700,1.156288,0.108124
std,0.915311,0.823779,1.756062,0.758019,865.735577,0.750806,1.551451,0.804260,0.885587,4.669174,3.103909,1.167642,0.329821,1.090109,1.093479,39.108967,2.202001,0.823781,0.242392
min,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.600397,1.000000,-2219.000000,-52.000000,0.000000,0.000000
25%,1.000000,2.000000,2.000000,0.000000,45.000000,1.000000,1.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,7.187260,2.000000,15.000000,0.000000,0.666667,0.000000
50%,2.000000,2.000000,3.000000,0.000000,45.000000,2.000000,2.000000,3.000000,3.000000,8.000000,5.000000,3.000000,1.000000,7.860814,2.000000,33.000000,0.000000,1.000000,0.000000
75%,3.000000,3.000000,4.000000,0.000000,45.000000,2.000000,3.000000,3.000000,3.000000,8.000000,7.000000,4.000000,1.000000,8.419144,3.000000,82.000000,2.000000,1.333333,0.000000
max,3.000000,7.000000,32.000000,13.000000,4995.000000,3.000000,7.000000,6.000000,4.000000,38.000000,13.000000,24.000000,2.000000,10.816652,26.000000,126.000000,50.000000,24.000000,8.000000


In [0]:
memberTrain = df['memberid'].value_counts()
memberFeatureTrain = [0]*len(df)

for i in range(len(df)):
    memberFeatureTrain[i] = memberTrain[df.loc[i, 'memberid']]

df['member_feature'] = memberFeatureTrain

In [0]:
df['Persontravellingid'] = df['persontravellingid'].replace([45, 47, 46, 4752, 4753, 4995], [0, 1, 2, 3, 4, 5])
df['State_code_residence'] = df['state_code_residence'].fillna(0)
df['Member_age_buckets'] = df['member_age_buckets'].replace(['D', 'E', 'F', 'C', 'H', 'G', 'B', 'I', 'A', 'J'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
df['Cluster_code'] = df['cluster_code'].replace(['F', 'E', 'D', 'C', 'B', 'A'], [5, 4, 3, 2, 1, 0])
df['Reservationstatusid_code'] = df['reservationstatusid_code'].replace(['A', 'B', 'C', 'D'], [0, 1, 2, 0])

resortCount=df['resort_id'].value_counts()
df['Resort_code'] = df['resort_id'].replace(resortCount.index.tolist(), list(range(0, 32)))
df = df.drop(['persontravellingid','state_code_residence','member_age_buckets','cluster_code','reservationstatusid_code'],1)

In [23]:
num_data = df[[ 'adultsp', 'childrenp', 'numberofadults', 'numberofchildren', 'stay', 'planning', 'extradays', 'total_pax','amount_spent_per_room_night_scaled']]
cat_data = df[['channel_code','main_product_code', 'Persontravellingid', 'resort_region_code','resort_type_code', 'season_holidayed_code', 'State_code_residence', 'state_code_resort','room_type_booked_code',
              'Member_age_buckets', 'booking_type_code', 'Cluster_code' ]]
#drop(['numberofadults', 'numberofchildren','stay', 'total_pax','amount_spent_per_room_night_scaled','state_code_residence'],1)
num_data.describe()

,adultsp,childrenp,numberofadults,numberofchildren,stay,planning,extradays,total_pax,amount_spent_per_room_night_scaled
count,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000
mean,1.156288,0.108124,3.275361,0.362696,2.466646,46.084754,1.271700,3.193519,7.720801
std,0.823781,0.242392,1.756062,0.758019,1.093479,39.108967,2.202001,1.167642,1.090109
min,0.000000,0.000000,0.000000,0.000000,1.000000,-2219.000000,-52.000000,1.000000,1.600397
25%,0.666667,0.000000,2.000000,0.000000,2.000000,15.000000,0.000000,2.000000,7.187260
50%,1.000000,0.000000,3.000000,0.000000,2.000000,33.000000,0.000000,3.000000,7.860814
75%,1.333333,0.000000,4.000000,0.000000,3.000000,82.000000,2.000000,4.000000,8.419144
max,24.000000,8.000000,32.000000,13.000000,26.000000,126.000000,50.000000,24.000000,10.816652


In [24]:
from sklearn.preprocessing import MinMaxScaler
#num_data= np.sqrt(num_data)

scaler = MinMaxScaler()
num_data[num_data.columns] = scaler.fit_transform(num_data[num_data.columns])
num_data.describe()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,adultsp,childrenp,numberofadults,numberofchildren,stay,planning,extradays,total_pax,amount_spent_per_room_night_scaled
count,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000
mean,0.048179,0.013515,0.102355,0.027900,0.058666,0.965921,0.522272,0.095370,0.664088
std,0.034324,0.030299,0.054877,0.058309,0.043739,0.016678,0.021588,0.050767,0.118281
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.027778,0.000000,0.062500,0.000000,0.040000,0.952665,0.509804,0.043478,0.606197
50%,0.041667,0.000000,0.093750,0.000000,0.040000,0.960341,0.509804,0.086957,0.679280
75%,0.055556,0.000000,0.125000,0.000000,0.080000,0.981237,0.529412,0.130435,0.739861
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
for i in cat_data.columns:
   print('column:',i,'len:',len(set(cat_data[i])))

column: channel_code len: 3
column: main_product_code len: 5
column: Persontravellingid len: 6
column: resort_region_code len: 3
column: resort_type_code len: 7
column: season_holidayed_code len: 4
column: State_code_residence len: 37
column: state_code_resort len: 11
column: room_type_booked_code len: 6
column: Member_age_buckets len: 10
column: booking_type_code len: 2
column: Cluster_code len: 6


BINARY ENCODING

In [0]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.7MB/s 


In [0]:
cat_dat = cat_data.copy()

import category_encoders as ce

encoder = ce.BinaryEncoder()
cat_dat = encoder.fit_transform(cat_dat)

cat_dat.head()

,channel_code,main_product_code,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,season_holidayed_code,state_code_resort,member_age_buckets_0,member_age_buckets_1,...,member_age_buckets_3,member_age_buckets_4,booking_type_code,cluster_code_0,cluster_code_1,cluster_code_2,cluster_code_3,reservationstatusid_code_0,reservationstatusid_code_1,reservationstatusid_code_2
0,3,1,46,3,3,3,2.0,3,0,0,...,0,1,1,0,0,0,1,0,0,1
1,1,1,46,3,3,4,2.0,5,0,0,...,0,1,1,0,0,0,1,0,1,0
2,1,1,47,1,5,4,2.0,1,0,0,...,0,1,1,0,0,1,0,0,1,0
3,1,1,46,2,2,3,2.0,2,0,0,...,0,1,1,0,0,1,1,0,1,0
4,1,1,46,2,2,4,2.0,2,0,0,...,0,1,1,0,0,1,1,0,1,0


ONE HOT ENCODING


In [26]:
cat_data=pd.get_dummies(cat_data.astype('category'))
cat_data.describe()

,channel_code_1,channel_code_2,channel_code_3,main_product_code_1,main_product_code_2,main_product_code_3,main_product_code_4,main_product_code_7,Persontravellingid_0,Persontravellingid_1,...,Member_age_buckets_8,Member_age_buckets_9,booking_type_code_1,booking_type_code_2,Cluster_code_0,Cluster_code_1,Cluster_code_2,Cluster_code_3,Cluster_code_4,Cluster_code_5
count,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.00000,341424.000000,341424.000000,341424.000000,341424.000000,...,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000
mean,0.472213,0.152763,0.375023,0.169355,0.520218,0.25261,0.054533,0.003283,0.782918,0.092495,...,0.020661,0.004036,0.875791,0.124209,0.086725,0.110329,0.110739,0.163676,0.175237,0.353294
std,0.499228,0.359760,0.484130,0.375066,0.499592,0.43451,0.227067,0.057206,0.412259,0.289724,...,0.142245,0.063402,0.329821,0.329821,0.281432,0.313300,0.313809,0.369982,0.380170,0.477994
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,1.000000,1.00000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
print(cat_data.shape)
for i in cat_data.columns:
  if (cat_data[i] == 1).sum() < 5000:
    cat_data.drop([i], axis=1, inplace= True)
    
print(cat_data.shape)

(341424, 100)
(341424, 66)


In [28]:
print(cat_data.shape)
for i in cat_data.columns:
  if cat_data[i].corr(num_data['amount_spent_per_room_night_scaled']) < 0.003:
      cat_data.drop([i], axis=1, inplace= True)
    
print(cat_data.shape)

(341424, 66)
(341424, 27)


In [68]:
df.corrwith(df['amount_spent_per_room_night_scaled'])

channel_code                          0.088124
main_product_code                    -0.081229
numberofadults                        0.150047
numberofchildren                      0.047827
persontravellingid                   -0.040716
resort_region_code                   -0.074258
resort_type_code                     -0.024186
room_type_booked_code                -0.050563
season_holidayed_code                -0.082181
state_code_residence                 -0.026229
state_code_resort                     0.033793
total_pax                             0.143138
booking_type_code                     0.020525
amount_spent_per_room_night_scaled    1.000000
stay                                 -0.198800
adultsp                               0.048417
childrenp                             0.033096
dtype: float64

In [30]:
correlation_dataframe = cat_data.copy()
correlation_dataframe.corrwith(df['amount_spent_per_room_night_scaled'])

channel_code_2               0.023089
channel_code_3               0.074726
main_product_code_1          0.059578
main_product_code_2          0.019785
Persontravellingid_0         0.112267
resort_region_code_1         0.048302
resort_region_code_2         0.016558
resort_type_code_1           0.045968
resort_type_code_4           0.005359
resort_type_code_5           0.045353
season_holidayed_code_1.0    0.072610
season_holidayed_code_2.0    0.029889
State_code_residence_1.0     0.027105
State_code_residence_2.0     0.043009
State_code_residence_10.0    0.017863
state_code_resort_1          0.025144
state_code_resort_6          0.069844
state_code_resort_7          0.048528
state_code_resort_11         0.017515
room_type_booked_code_1      0.017035
room_type_booked_code_2      0.061283
Member_age_buckets_0         0.027646
Member_age_buckets_1         0.031175
Member_age_buckets_3         0.003002
booking_type_code_2          0.020525
Cluster_code_2               0.048528
Cluster_code

**MODEL**

In [0]:
train=pd.concat([cat_data, num_data], axis = 1)
Y =train['amount_spent_per_room_night_scaled']
X =train.drop(['amount_spent_per_room_night_scaled'],axis=1)

In [32]:
X.isnull().sum()
#X["season_holidayed_code"].value_counts()

channel_code_2               0
channel_code_3               0
main_product_code_1          0
main_product_code_2          0
Persontravellingid_0         0
resort_region_code_1         0
resort_region_code_2         0
resort_type_code_1           0
resort_type_code_4           0
resort_type_code_5           0
season_holidayed_code_1.0    0
season_holidayed_code_2.0    0
State_code_residence_1.0     0
State_code_residence_2.0     0
State_code_residence_10.0    0
state_code_resort_1          0
state_code_resort_6          0
state_code_resort_7          0
state_code_resort_11         0
room_type_booked_code_1      0
room_type_booked_code_2      0
Member_age_buckets_0         0
Member_age_buckets_1         0
Member_age_buckets_3         0
booking_type_code_2          0
Cluster_code_2               0
Cluster_code_4               0
adultsp                      0
childrenp                    0
numberofadults               0
numberofchildren             0
stay                         0
planning

In [33]:
n = X.shape[1]
gbr_reg = GradientBoostingRegressor(max_depth=3, n_estimators=n,  
          criterion='mse', random_state=42) 
gbr_reg.fit(X,Y)
y_hat_gbr= gbr_reg.predict(X)

mse_gbr = mean_squared_error(Y, y_hat_gbr)
rmse_gbr = np.sqrt(mse_gbr)
rmse_gbr

0.10960553322746933

In [34]:
model = RandomForestRegressor(max_depth=3, n_estimators=150, random_state=42)
model.fit(X, Y)

y_hat= model.predict(X)

mse = mean_squared_error(Y, y_hat)
rmse = np.sqrt(mse)
rmse
#test_predicted = pd.DataFrame()
#test_predicted['Id'] = test_data.Id
#test_predicted['Prediction'] = model.predict(test_data.drop('Id', axis=1))
#test_predicted.to_csv('submission-random-forest.csv', index=False)
#test_predicted.describe()

0.11190052488387797

In [35]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, Y)
y_hat= model.predict(X)

mse = mean_squared_error(Y, y_hat)
rmse = np.sqrt(mse)
rmse
#test_predicted = pd.DataFrame()
#test_predicted['Id'] = test_data.Id
#test_predicted['Prediction'] = model.predict(test_data.drop('Id', axis=1))
#test_predicted.to_csv('submission-logreg.csv', index=False)
#test_predicted.describe()

0.10982044773101043